## Load and transform simulation outputs.

In [1]:
from shared.preface import *

In [2]:
#? check the zeds and nums ordering
box_name = 'L025N752'
box_ver = 'DMONLY/SigmaConstant00'
out_dir = f'{box_name}/{box_ver}'
nums_snaps = np.load(f'{out_dir}/nums_snaps.npy')
zeds = np.load(f'{out_dir}/zeds_snaps.npy')
zint = np.load(f'{out_dir}/z_int_steps.npy')
sint = np.load(f'{out_dir}/s_int_steps.npy')
print(nums_snaps)
print(zeds)
print(zint)
print(sint)

['0012' '0013' '0014' '0015' '0016' '0017' '0018' '0019' '0020' '0021'
 '0022' '0023' '0024' '0025' '0026' '0027' '0028' '0029' '0030' '0031'
 '0032' '0033' '0034' '0035' '0036']
[3.81000000e+00 3.43000000e+00 3.09000000e+00 2.79000000e+00
 2.52000000e+00 2.28000000e+00 2.06000000e+00 1.86000000e+00
 1.68000000e+00 1.51000000e+00 1.36000000e+00 1.21000000e+00
 1.08000000e+00 9.60000000e-01 8.50000000e-01 7.40000000e-01
 6.40000000e-01 5.50000000e-01 4.60000000e-01 3.70000000e-01
 2.90000000e-01 2.10000000e-01 1.40000000e-01 7.00000000e-02
 2.22044605e-16]
[0.00000000e+00 3.82232399e-03 7.79074959e-03 1.19108613e-02
 1.61884570e-02 2.06295562e-02 2.52404087e-02 3.00275029e-02
 3.49975753e-02 4.01576200e-02 4.55148984e-02 5.10769492e-02
 5.68515997e-02 6.28469760e-02 6.90715150e-02 7.55339761e-02
 8.22434534e-02 8.92093886e-02 9.64415845e-02 1.03950218e-01
 1.11745856e-01 1.19839469e-01 1.28242446e-01 1.36966612e-01
 1.46024243e-01 1.55428087e-01 1.65191376e-01 1.75327850e-01
 1.85851772

In [4]:
from shared.functions import read_DM_halo_index

with h5py.File(f'{out_dir}/MergerTree.hdf5') as tree:
    prog_IDs = tree['Assembly_history/Progenitor_index'][28,:]
    prog_IDs_np = np.array(np.expand_dims(prog_IDs, axis=1), dtype=int)

# The prog_IDs_np array is already from z=4 to z=0.

with open(f'{out_dir}/box_parameters.yaml', 'r') as file:
    box_setup = yaml.safe_load(file)

box_file_dir = box_setup['File Paths']['Box File Directory']
print(box_file_dir, type(box_file_dir))

print(prog_IDs_np)

/projects/0/einf180/Tango_sims/L025N752/DMONLY/SigmaConstant00 <class 'str'>
[[    43]
 [    44]
 [    55]
 [    60]
 [    62]
 [    69]
 [    71]
 [    74]
 [    66]
 [    65]
 [    60]
 [    67]
 [    72]
 [    75]
 [    80]
 [    98]
 [    99]
 [    87]
 [    85]
 [    76]
 [    68]
 [    67]
 [    68]
 [    61]
 [333410]]


In [ ]:
# def test_matching(snap, prog_ID):

#     prog_ID = int(prog_ID)
#     IDname = f'TEST_{snap}_{prog_ID}'
#     print(IDname)
    # read_DM_halo_index(
    #     snap, prog_ID, IDname, box_file_dir, out_dir
    # )
    # DM_raw = np.load(f'{out_dir}/DM_pos_{IDname}.npy')
    # DM_particles = len(DM_raw)
    # del DM_raw

    # np.save(f'{out_dir}/DM_particles_{snap}.npy', DM_particles)

# test_matching(nums_snaps[::-1][0], prog_IDs_np[0])

# with ProcessPoolExecutor(25) as ex:
#     ex.map(test_matching, nums_snaps[::-1][:2], prog_IDs_np[:2])

# DM_parts_snap = np.array(
#     [np.load(f'{out_dir}/DM_particles_{snap}.npy') for snap in nums_snaps[::-1][:2]]
# )
# print(DM_parts_snap)

In [ ]:
from shared.preface import *
import shared.functions as fct

def transform_simulation_outputs(box_name, box_ver, sim_fullname):
    """
    Input sim_fullname specifies the simulation mode with additional information: e.g. "single_halos_40k_test1" or "spheres_5shells_10k_final".
    """

    # Path to simulation outputs.
    out_dir = f'{box_name}/{box_ver}/{sim_fullname}'

    if 'single_halos' or 'spheres' in sim_fullname:
        # Load neutrino vectors (positions and velocities) of the 10k batches, and concatenate them into a single array.
        neutrino_vectors = np.array([[[]]])  # create empty (1,1,1) array

        batch_paths = glob.glob(f'{out_dir}/neutrino_vectors_batch*.npy')
        for batch_path in batch_paths:
            batch_vectors = np.load(batch_path)
            neutrino_vectors = np.concatenate(
                (neutrino_vectors, batch_vectors), axis=1
            )

        # Calculate overdensities.
        overdensities = np.load(f'{out_dir}/number_densities.npy')/N0

        return overdensities, neutrino_vectors
    
    else:
        # Load angle pairs and calculate overdensities for the all_sky mode.
        all_sky_output = np.load(f'{out_dir}/number_densities.npy')
        angle_pairs = all_sky_output[:, :2]
        overdensities = all_sky_output[:, 2:]/N0

        return overdensities, angle_pairs
    

box_name = 'L025N752'
box_ver = 'DMONLY/SigmaConstant00'
sim_fullname = 'single_halos_10k'
sim_arrays = transform_simulation_outputs(box_name, box_ver, sim_fullname)

## Analyze and plot simulation outputs.

In [ ]:
def analyze_simulation_outputs(
        sim_arrays_to_analyze, 
        box_name, box_ver, sim_fullname,
        plots_to_make
    ):

    # Figure directory.
    if not os.path.exists('figures'):
        os.makedirs('figures')
    fig_dir = f'figures/{box_name}/{box_ver}/{sim_fullname}'

    savefig_args = dict(
        bbox_inches='tight'
    )


    ### ================================== ###
    ### Figure of merit: overdensity band. ###
    ### ================================== ###

    fig, ax = plt.subplots(1,1)

    # Numerical simulation.
    if etas_sim.ndim <= 1:
        ax.plot(
            m_nu_range*1e3, (etas_sim-1), color='blue', 
            label='medians'
        )
    else:
        nus_median = np.median(etas_sim, axis=0)
        nus_perc2p5 = np.percentile(etas_sim, q=2.5, axis=0)
        nus_perc97p5 = np.percentile(etas_sim, q=97.5, axis=0)
        nus_perc16 = np.percentile(etas_sim, q=16, axis=0)
        nus_perc84 = np.percentile(etas_sim, q=84, axis=0)
        ax.plot(
            m_nu_range*1e3, (nus_median-1), color='blue', 
            label='Halo sample medians'
        )
        ax.fill_between(
            m_nu_range*1e3, (nus_perc2p5-1), (nus_perc97p5-1), 
            color='blue', alpha=0.2, label='2.5-97.5 %'
        )
        ax.fill_between(
            m_nu_range*1e3, (nus_perc16-1), (nus_perc84-1), 
            color='blue', alpha=0.3, label='16-84 %'
        )
    plt.savefig(fig_dir, **savefig_args)